In [1]:
import pandas as pd

df_path = "/home/jorge/data/awto_mle_challenge/data/wind_power_generation.csv"
df = pd.read_csv(df_path)

# Rename "Unnamed: 0" to "ds"
df.rename(columns={"Unnamed: 0": "ds"}, inplace=True)
df

,ds,ActivePower,AmbientTemperatue,BearingShaftTemperature,Blade1PitchAngle,Blade2PitchAngle,Blade3PitchAngle,ControlBoxTemperature,GearboxBearingTemperature,GearboxOilTemperature,...,GeneratorWinding2Temperature,HubTemperature,MainBoxTemperature,NacellePosition,ReactivePower,RotorRPM,TurbineStatus,WTG,WindDirection,WindSpeed
0,2017-12-31 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G01,NaN,NaN
1,2017-12-31 00:10:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G01,NaN,NaN
2,2017-12-31 00:20:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G01,NaN,NaN
3,2017-12-31 00:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G01,NaN,NaN
4,2017-12-31 00:40:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G01,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118219,2020-03-30 23:10:00+00:00,70.044465,27.523741,45.711129,1.515669,1.950088,1.950088,0.0,59.821165,55.193793,...,58.148777,39.008931,36.476562,178.0,13.775785,9.234004,2.0,G01,178.0,3.533445
118220,2020-03-30 23:20:00+00:00,40.833474,27.602882,45.598573,1.702809,2.136732,2.136732,0.0,59.142038,54.798545,...,57.550367,39.006759,36.328125,178.0,8.088928,9.229370,2.0,G01,178.0,3.261231
118221,2020-03-30 23:30:00+00:00,20.777790,27.560925,45.462045,1.706214,2.139664,2.139664,0.0,58.439439,54.380456,...,57.099335,39.003815,36.131944,178.0,4.355978,9.236802,2.0,G01,178.0,3.331839
118222,2020-03-30 23:40:00+00:00,62.091039,27.810472,45.343827,1.575352,2.009781,2.009781,0.0,58.205413,54.079014,...,56.847239,39.003815,36.007805,190.0,12.018077,9.237374,2.0,G01,190.0,3.284468


In [2]:
# Preprocess

# Convert ds to datetime
df["ds"] = pd.to_datetime(df["ds"]).dt.tz_localize(None)


# Remove all NaN in ActivePower column
df = df.dropna(subset=["ActivePower"])

# Convert NaN in another columns to 0
#df = df.fillna(0)

# Delete WTG column
df = df.drop(columns=["WTG"])

df.reset_index(drop=True, inplace=True)

In [3]:
import numpy as np
import matplotlib.pyplot as plt

from darts import TimeSeries

series = TimeSeries.from_dataframe(df, 
                                   time_col="ds", 
                                   value_cols=df.columns.tolist()[1:], 
                                   fill_missing_dates=True,
                                   freq = "10T", #10 minutes
                                   fillna_value = 0,
                                   )

# Split the series into train and eval
train, val = series.split_before(0.8)


In [4]:
from darts.metrics import mape, mae, rmse

models = []

In [5]:
import optuna
import pandas as pd
from darts.models import TCNModel
from darts.metrics import mae, rmse

# Define the objective function to optimize
def objective(trial):
    # Define the hyperparameters to search over
    input_chunk_length = trial.suggest_int('input_chunk_length', 12, 48)
    num_layers = trial.suggest_int('num_layers', 2, 12)
    num_filters = trial.suggest_int('num_filters', 16, 128)
    n_epochs = trial.suggest_int('n_epochs', 10, 100)
    dropout = trial.suggest_uniform('dropout', 0.0, 0.5)

    # Create the TCN model with the current hyperparameters
    model = TCNModel(input_chunk_length=input_chunk_length,
                     output_chunk_length=1,
                     num_layers=num_layers,
                     num_filters=num_filters,
                     n_epochs=n_epochs,
                     dropout=dropout,
                     random_state=13)

    # Fit the model on the training set
    model.fit(train)

    # Make predictions on the validation set
    pred_val = model.predict(len(val), verbose=True)

    # Evaluate the model's performance using root mean squared error
    rmse_score = rmse(val["ActivePower"], pred_val["ActivePower"])

    return rmse_score

# Set up the optuna study
study = optuna.create_study(direction='minimize')

# Run the optimization
study.optimize(objective, n_trials=2)

# Create an empty DataFrame to store all results
results = pd.DataFrame(columns=['input_chunk_length',
                                'num_layers',
                                'num_filters',
                                'n_epochs',
                                'dropout',
                                'MAE',
                                'RMSE'])

# Loop over all trials and store the hyperparameters and results in the DataFrame
for trial in study.trials:
    # Get the hyperparameters and their corresponding score
    params = trial.params
    rmse_score = trial.value

    # Create the TCN model using the hyperparameters
    model = TCNModel(input_chunk_length=params['input_chunk_length'],
                     output_chunk_length=1,
                     num_layers=params['num_layers'],
                     num_filters=params['num_filters'],
                     n_epochs=params['n_epochs'],
                     dropout=params['dropout'],
                     random_state=13)

    # Fit the model on the training set
    model.fit(train)

    # Make predictions on the validation set
    pred_val = model.predict(len(val), verbose=True)

    # Evaluate the model's performance using both mean absolute error and root mean squared error
    mae_score = mae(val["ActivePower"], pred_val["ActivePower"])
    rmse_score = rmse(val["ActivePower"], pred_val["ActivePower"])

    # Add the hyperparameters and results to the DataFrame
    results = results.append({'input_chunk_length': params['input_chunk_length'],
                              'num_layers': params['num_layers'],
                              'num_filters': params['num_filters'],
                              'n_epochs': params['n_epochs'],
                              'dropout': params['dropout'],
                              'MAE': mae_score,
                              'RMSE': rmse_score}, ignore_index=True)

# Print the results
print(results)

[I 2023-03-15 16:17:04,895] A new study created in memory with name: no-name-b8838bb8-d45d-4458-867c-7420194575cb
/tmp/ipykernel_225594/1293849618.py:13: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  dropout = trial.suggest_uniform('dropout', 0.0, 0.5)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type              | Params
----------------------------------------------------
0 | criterion     | MSELoss           | 0     
1 | train_metrics | MetricCollection  | 0     
2 | val_metrics   | MetricCollection  | 0     
3 | dropout       | MonteCarloDropout | 0     
4 | res_blocks    | ModuleList        | 678 K 
----------------------------------------------

Training: 0it [00:00, ?it/s]